# Using LLMs

This Jupyter Notebook demonstrates the use of various Large Language Model (LLM) providers and tools for natural language processing tasks. It showcases how to interact with LLMs using the `crewai` library (which is used by Agentics), parse structured outputs with Pydantic models, and explore available LLM connections. The notebook provides practical examples for calling different LLMs, formatting responses, and integrating with external providers such as OpenAI and IBM WatsonX.

In [ ]:
! uv pip install agentics-py


import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

## Connect to your own LLM provider
Agentics uses CrewAI wrappers for main LLM providers. You can initialize your LLM as follows.
[find out more](https://docs.crewai.com/en/concepts/llms)

In [ ]:
from crewai import LLM

# pick a provider (openai, anthropic, groq, etc.) - see crewai docs for details
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,  # Adjust based on task
    max_tokens=4096,  # Set based on output needs
    timeout=300,
)  # Longer timeout for complex tasks

print(llm)

## Perform Simple LLM call

Once an LLM is instatiated, you can perform LLM calls

In [ ]:
print(llm.call("where is the Eiffel Tower?"))

## Perform Structured Call

LLMs can generate structured objects given a pydantic schema if you instantiate them accordingly

In [ ]:
from pydantic import BaseModel


class Answer(BaseModel):
    short_answer: str
    detailed_answer: str
    confidence: float


struct_llm = LLM(model="gemini/gemini-2.0-flash", response_format=Answer)

print(struct_llm.call("Why is the sky blue?"))

## You can also use structured decoding for information extraction

In [ ]:
import requests


class Person(BaseModel):
    name: str
    city: str
    occupation: str


class Book(BaseModel):
    title: str
    publisher: str
    year: int
    author: Person


class InformationExtraction(BaseModel):
    books: list[Book]
    people: list[Person]


person_llm = LLM(model="gemini/gemini-2.0-flash", response_format=InformationExtraction)
print(person_llm.call(requests.get("https://iep.utm.edu/wittgens/").text))

## Using Agentics Predefined LLMs

Agentics has the following LLM handles: watsonx_llm, openai_llm, gemini_llm
You can directly import and use them as defaults

In [ ]:
from agentics import AG

print(AG.get_llm_provider("list"))

llm = AG.get_llm_provider()  ## get the default LLM provider from the available ones
print(llm.call("Where is Rome?"))

llm = AG.get_llm_provider(
    "gemini"
)  ## get a specific LLM provider from the available ones,change "gemini" with "openai", "watsonx", etc. as needed
print(llm.call("Where is Rome?"))

## Homework

Play with different data models on your favourite domain (e.g. stocks) providing relevant data. Enjoy tructured decoding magic.